<a href="https://colab.research.google.com/github/LuisFerRosas/ia3/blob/main/Copia_de_Final_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/Proyecto_ia3

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/Proyecto_ia3


In [ ]:
import json

In [ ]:
# !git clone https://github.com/LuisFerRosas/ia3.git

In [ ]:
with open("./ia3/datos/vocabulario2.json", "r") as fp:
        vocabPart = json.load(fp)

len(vocabPart)

233

In [ ]:
cd ia3/

/gdrive/My Drive/Colab Notebooks/Proyecto_ia3/ia3


In [ ]:
# !git config --global user.name "LuisFerRosas"
# !git config --global user.email luis3658duran@gmail.com

In [ ]:
# !git config --list

In [ ]:
# !git status

In [ ]:
# !git pull

In [ ]:
!pip install -r requirements.txt

In [ ]:
# !python train.py

In [ ]:
# !python prepare_data.py

In [ ]:
# !tensorboard --logdir=runs/transformer

In [ ]:
from preprocess import get_dataset, DataLoader, collate_fn_transformer
from network import *
from utils import load_checkpoint
from tensorboardX import SummaryWriter
import os
from tqdm import tqdm
import hyperparams as hp

In [ ]:
load_model=True
# load_model=False
NHEAD=8
num_decoder_layer=6
num_encoder_layer=6
vocab=len(vocabPart)
post_mel_size=2000
num_hidden=hp.hidden_size
maxlen=len(vocabPart)
NUM_EPOCHS=1001
inicioEpoch=651
batch_size=5
save_step=50
checkpoint_path='../checkpoint'
checkpoint_path_load = '../checkpoint/checkpoint_transformer_650.pth.tar'
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda', index=0)

In [ ]:
modelo=ModelTransformer(hp.embedding_size,NHEAD,num_decoder_layer,vocab,post_mel_size,num_hidden,num_encoder_layer,maxlen)

# print(modelo)

In [ ]:
for p in modelo.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
modelo=modelo.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(
    modelo.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(modelo):,} parametros entrenables')

El modelo tiene 26,813,929 parametros entrenables


In [ ]:
if load_model:
    load_checkpoint(torch.load(checkpoint_path_load), modelo, optimizer)


=> Loading checkpoint


In [ ]:
def main():
  dataset=get_dataset()


  modelo.train()
  writer = SummaryWriter("runs/tranformer")
  
  estep=0
  for epoch in range(inicioEpoch,NUM_EPOCHS):
      dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn_transformer, drop_last=True,shuffle=True )
      pbar = tqdm(dataloader)
      losses = []
      for i, data in enumerate(pbar):
          estep=estep+1
          pbar.set_description("Processing at epoch %d"%epoch)
          character,  mel_input, pos_text, pos_mel, _ = data
          character = character.to(DEVICE)            
          mel_input = mel_input.to(DEVICE)
          pos_text = pos_text.to(DEVICE)            
          pos_mel = pos_mel.to(DEVICE)
          
          
          output=modelo(character,mel_input,pos_text,pos_mel)
          # print(output)
          if estep==1:
              writer.add_graph(modelo ,(character,mel_input,pos_text,pos_mel))
              # writer.add_graph(modelo )
          
          
          # print("output modelo...."+str(output.shape))
          # print("output trasformado..."+str(output.reshape(-1, output.shape[-1]).shape))
          # print("caracter ......"+str(character.reshape(-1).shape))
          optimizer.zero_grad()
          loss = loss_fn(output.reshape(-1, output.shape[-1]), character.reshape(-1))
          output=output.transpose(0,1)
          loss2=loss.item()
          writer.add_scalar("loss :",loss2 ,estep)
          # print("/////////////////")
          # print(np.argmax(output[0].detach().numpy(),axis=1))
          print("loss..........."+str(loss2))
          # print("Epoch.........."+str(epoch))
          
          loss.backward()
          optimizer.step()
          losses.append(loss.item())
      writer.add_scalar("loss2 :",(sum(losses)/len(losses)) ,epoch)
      losses=[]
      if epoch % save_step==0:
          t.save({'model':modelo.state_dict(),
                  'optimizer':optimizer.state_dict()},
                              os.path.join(checkpoint_path,'checkpoint_transformer_%d.pth.tar' % epoch))
  writer.close()


In [ ]:

main()

Processing at epoch 651:   0%|          | 0/54 [00:06<?, ?it/s]

loss...........3.5473990440368652


Processing at epoch 651:   2%|▏         | 1/54 [00:15<09:27, 10.72s/it]

loss...........3.6442084312438965


Processing at epoch 651:   4%|▎         | 2/54 [00:24<08:25,  9.71s/it]

...

loss...........3.632007598876953


Processing at epoch 651:  35%|███▌      | 19/54 [02:26<03:53,  6.67s/it]

loss...........3.8639466762542725


Processing at epoch 651:  37%|███▋      | 20/54 [02:33<03:50,  6.79s/it]

RuntimeError: ignored